In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
import warnings
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
warnings.filterwarnings('ignore')

#### 用convert_dtypes()读取数据

In [2]:
train = pd.read_csv('./input/train.csv').convert_dtypes(convert_string=False)
testa = pd.read_csv('./input/testA.csv').convert_dtypes(convert_string=False)

In [3]:
# train.dtypes Int

#### issueDate

#### 这里对了吗？？？datetime? string?

In [4]:
#转化成时间格式，这里暂时不新增，感觉意义不明确，但是查了确实train的最小日期是2007-6-1
for data in [train, testa]:
    data['issueDate'] = pd.to_datetime(data['issueDate'],format='%Y-%m-%d')
    startdate = datetime.datetime.strptime('2007-06-01', '%Y-%m-%d')
    # 构造时间特征
    data['issueDateDT'] = data['issueDate'].apply(lambda x: x-startdate).dt.days

In [5]:
# train.dtypes   # issueDate datetime64[ns]

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Data columns (total 48 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   id                  800000 non-null  Int64         
 1   loanAmnt            800000 non-null  Int64         
 2   term                800000 non-null  Int64         
 3   interestRate        800000 non-null  float64       
 4   installment         800000 non-null  float64       
 5   grade               800000 non-null  object        
 6   subGrade            800000 non-null  object        
 7   employmentTitle     799999 non-null  Int64         
 8   employmentLength    753201 non-null  object        
 9   homeOwnership       800000 non-null  Int64         
 10  annualIncome        800000 non-null  float64       
 11  verificationStatus  800000 non-null  Int64         
 12  issueDate           800000 non-null  datetime64[ns]
 13  isDefault           800000 no

#### employmentLength的 years删掉，10+ <1转

In [7]:
def employmentLength_to_int(s):
    if pd.isnull(s):
        return s
    else:
        return np.int8(s.split()[0])
for data in [train, testa]:
    data['employmentLength'].replace(to_replace='10+ years', value='10 years', inplace=True)
    data['employmentLength'].replace('< 1 year', '0 years', inplace=True)
    data['employmentLength'] = data['employmentLength'].apply(employmentLength_to_int)

In [8]:
train['employmentLength'].value_counts(dropna=False).sort_index()

0.0      64237
1.0      52489
2.0      72358
3.0      64152
4.0      47985
5.0      50102
6.0      37254
7.0      35407
8.0      36192
9.0      30272
10.0    262753
NaN      46799
Name: employmentLength, dtype: int64

#### 这里还是float要转吗，一会再说吧

In [9]:
# train['employmentLength'].astype('Int') # 没这种写法

In [10]:
# test[]

#### earliesCreditLine 这个要怎么弄

In [11]:
pd.set_option('display.max_rows', None)

In [12]:
train['earliesCreditLine'].sample(20)

143859    Mar-1996
334713    Dec-1976
127251    Apr-2002
302509    Jun-2001
207638    Dec-1998
506933    Nov-1993
227379    Aug-2005
655268    Jun-2007
54332     Mar-2008
557360    Jan-2000
125725    Oct-1994
60320     Apr-1999
182458    Nov-2001
488207    Apr-1987
423808    Jul-2000
118090    May-1995
333611    Mar-1998
424819    Mar-2008
555609    Dec-2011
232220    Apr-2008
Name: earliesCreditLine, dtype: object

In [13]:
for data in [train, testa]:
    data['earliesCreditLine'] = pd.to_datetime(data['earliesCreditLine'],format='%b-%Y')

In [15]:
pd.set_option('display.max_columns', None)
train.describe(include="all")

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,issueDate,isDefault,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,policyCode,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,issueDateDT
count,800000.000000,800000.000000,800000.000000,800000.000000,800000.000000,800000,800000,799999.000000,753201.000000,800000.000000,8.000000e+05,800000.000000,800000,800000.000000,800000.000000,799999.000000,800000.000000,799761.000000,800000.000000,800000.000000,800000.000000,800000.000000,800000.000000,799595.000000,8.000000e+05,799469.000000,800000.000000,800000.000000,800000.000000,800000,799999.000000,800000.0,759730.000000,759730.000000,759730.000000,759730.000000,766761.000000,759730.000000,759730.000000,759730.000000,759729.000000,759730.000000,766761.000000,730248.000000,759730.000000,759730.000000,759730.000000,800000.000000
unique,NaN,NaN,NaN,NaN,NaN,7,35,NaN,NaN,NaN,NaN,NaN,139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,B,C1,NaN,NaN,NaN,NaN,NaN,2016-03-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001-08-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,233690,50763,NaN,NaN,NaN,NaN,NaN,29066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007-06-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1944-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,399999.500000,14416.818875,3.482745,13.238391,437.947723,NaN,NaN,72005.351714,5.965205,0.614213,7.613391e+04,1.009683,NaN,0.199513,1.745982,258.535648,16.385758,18.284557,0.318239,696.204081,700.204226,11.598020,0.214915,0.134163,1.622871e+04,51.790734,24.998861,0.416953,0.019267,NaN,1754.113589,1.0,0.511932,3.642330,5.642648,5.642648,4.735641,8.107937,8.575994,8.282953,14.622488,5.592345,11.643896,0.000815,0.003384,0.089366,2.178606,2927.406960
std,230940.252013,8716.086178,0.855832,4.765757,261.460393,NaN,NaN,106585.640204,3.690361,0.675749,6.894751e+04,0.782716,NaN,0.399634,2.367453,200.037446,11.036679,11.150155,0.880325,31.865995,31.866674,5.475286,0.606467,0.377471,2.245802e+04,24.516126,11.999201,0.493055,0.137464,NaN,7941.474040,0.0,1.333266,2.246825,3.302810,3.302810,2.949969,4.799210,7.400536,4.561689,8.124610,3.216184,5.484104,0.030075,0.062041,0.509069,1.844377,591.067525
min,0.000000,500.000000,3.000000,5.310000,15.690000,NaN,NaN,0.000000,0.000000,0.000000,0.000000e+00,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,630.000000,634.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,2.000000,0.000000,0.000000,NaN,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,199999.750000,8000.000000,3.000000,9.750000,248.450000,NaN,NaN,427.000000,2.000000,0.000000,4.560000e+04,0.000000,NaN,0.000000,0.000000,103.000000,8.000000,11.790000,0.000000,670.000000,674.000000,8.000000,0.000000,0.000000,5.944000e+03,33.400000,16.000000,0.000000,0.000000,NaN,0.000000,1.0,0.000000,2.000000,3.000000,3.000000,3.000000,5.000000,4.000000,5.000000,9.000000,3.000000,8.000000,0.000000,0.000000,0.000000,1.000000,2587.000000


In [ ]:
# dw这里是提取年信息，我想利用原始信息，顺便还可以describe一下
# for data in [train, testa]:
#     data['earliesCreditLine'] = data['earliesCreditLine'].apply(lambda s: int(s[-4:]))

#### 压缩一下内存

In [ ]:
# ## Simple "Memory profilers" to see memory usage
# def get_memory_usage():
#     return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2) 
        
# def sizeof_fmt(num, suffix='B'):
#     for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
#         if abs(num) < 1024.0:
#             return "%3.1f%s%s" % (num, unit, suffix)
#         num /= 1024.0
#     return "%.1f%s%s" % (num, 'Yi', suffix)

In [ ]:
# ## Memory Reducer
# # :df pandas dataframe to reduce size             # type: pd.DataFrame()
# # :verbose                                        # type: bool
# def reduce_mem_usage(df, verbose=True):
#     numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
#     start_mem = df.memory_usage().sum() / 1024**2    
#     for col in df.columns:
#         col_type = df[col].dtypes
#         if col_type in numerics:
#             c_min = df[col].min()
#             c_max = df[col].max()
#             if str(col_type)[:3] == 'int':
#                 if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
#                     df[col] = df[col].astype(np.int8)
#                 elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
#                        df[col] = df[col].astype(np.int16)
#                 elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
#                     df[col] = df[col].astype(np.int32)
#                 elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
#                     df[col] = df[col].astype(np.int64)  
#             else:
#                 if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
#                     df[col] = df[col].astype(np.float16)
#                 elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
#                     df[col] = df[col].astype(np.float32)
#                 else:
#                     df[col] = df[col].astype(np.float64)    
#     end_mem = df.memory_usage().sum() / 1024**2
#     if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
#     return df

In [ ]:
# print("{:>20}: {:>8}".format('Original grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

这没写呢

'earliesCreditLine'不会处理先drop掉

#### 数据类型转换

In [ ]:
# for data in [train, testa]:
#     data['employmentTitle'] = data['employmentTitle'].astype('int32')

In [ ]:
# train['employmentTitle'] = train['employmentTitle'].astype('int32')

In [ ]:
# cat_fea = ['grade', 'subGrade', 'employmentTitle', 'verificationStatus', \
#           'purpose', 'postCode', 'regionCode','applicationType','initialListStatus', 'title', 'policyCode']

In [ ]:
# cols = ['id', 'loanAmnt', 'term', 'interestRate', 'installment', 'grade',
#        'subGrade', 'employmentTitle', 'employmentLength', 'homeOwnership',
#        'annualIncome', 'verificationStatus', 'issueDate', 'isDefault',
#        'purpose', 'postCode', 'regionCode', 'dti', 'detlinquency_2years',
#        'ficoRangeLow', 'ficoRangeHigh', 'openAcc', 'pubRec',
#        'pubRecBankruptcies', 'revolBal', 'revolUtil', 'totalAcc',
#        'initialListStatus', 'applicationType', 'earliesCreditLine', 'title',
#        'policyCode', 'n0', 'n1', 'n2', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9',
#        'n10', 'n11', 'n12', 'n13', 'n14']

'issueDate', 'isDefault', 'earliesCreditLine'

In [ ]:
# num_fea = [a for a in cols if a not in cat_fea]
# num_fea

In [ ]:
# ## 定义了一个统计函数，方便后续信息统计
# def Sta_inf(data):
#     print('_min',np.min(data))
#     print('_max:',np.max(data))
#     print('_mean',np.mean(data))
#     print('_ptp',np.ptp(data))
#     print('_std',np.std(data))
#     print('_var',np.var(data))